## Загрузка и предобработка файла

In [393]:
import pandas as pd

https://drive.google.com/file/d/1hAgbwKURl8yLgMDiTrB-T_V5tg3SqQTn/view?usp=sharing

In [394]:
#! gdown --id 1hAgbwKURl8yLgMDiTrB-T_V5tg3SqQTn


In [395]:
df = pd.read_csv('perechetka_gpzu.csv', sep=';')

In [396]:
df.head()

,number,type,tree,bush,diameter,height,description,action
0,1044,Береза,1.0,NaN,16,NaN,сухостой,Вырубить
1,1066,Береза,1.0,NaN,"18,46",NaN,2 ствола,Сохранить
2,1067,Сосна,1.0,NaN,66,NaN,NaN,Вырубить
3,1068,Сосна,1.0,NaN,48,NaN,многочисл.сухие ветви,Вырубить
4,1069,Береза,1.0,NaN,46,NaN,NaN,Вырубить


In [397]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   number       729 non-null    object 
 1   type         729 non-null    object 
 2   tree         631 non-null    float64
 3   bush         78 non-null     float64
 4   diameter     652 non-null    object 
 5   height       0 non-null      float64
 6   description  247 non-null    object 
 7   action       729 non-null    object 
dtypes: float64(3), object(5)
memory usage: 45.7+ KB


In [398]:
df['description'] = df.description.fillna('no info')

In [399]:
df['diameter'] = df.diameter.fillna(0)

In [400]:
df.diameter.unique()

array(['16', '18,46', '66', '48', '46', '58', '28,32', '12', '10', '8',
       '50', '36', 0, '44', '6', '52', '18', '1-2', '54', '76', '22',
       '8,8,12', '72', '56', '1', '1-4', '30', '64', '28', '36,58', '38',
       '40', '26', '18,42', '70', '14', '20', '54,66', '68', '60', '24',
       '84', '34', '18,20,30', '6,6', '74', '30,30', '42', '50-52', '88',
       '36,36', '32', '78', '62', '8,8', '86', '25', '10,50', '53',
       '10-12', '39', '18,32', '18,24', '46-48', '30-34', '63', '15',
       '16-24', '49', '19', '20-26', '52-56', '6-10', '11', '47', '21',
       '33', '6-8', '56-62', '36-54', '13', '48-52', '55', '24,28', '17',
       '16,20,24', '20,20,24', '44-50', '16-18', '32-34', '35', '34,36',
       '32-36', '31', '18-24', '8-12', '8,8,10'], dtype=object)

В столбцах tree и bush есть пропуски - это специфика данных, в tree отмечено количество только деревьев, bush соответственно кустов

In [401]:
df.tree.value_counts()

1.0    594
2.0     25
3.0      4
5.0      3
6.0      2
7.0      1
4.0      1
9.0      1
Name: tree, dtype: int64

In [402]:
df = df[~(df.diameter == '01.февр')]

In [403]:
df['tree'] = pd.to_numeric(df['tree'], errors='coerce')
df['bush'] = pd.to_numeric(df['bush'], errors='coerce')


In [404]:
display('Общее количество деревьев и кустарников')
df.groupby('action', as_index=False)[['tree', 'bush']].sum()

'Общее количество деревьев и кустарников'

,action,tree,bush
0,Вырубить,97.0,20.0
1,Сохранить,606.0,166.0


## Получение данных

Тут только вырубка (еще бывает пересадка и сохранение). Значит для расчетов нам нужны следующие данные: 
1. количество поросли и кустарников, 
2. количество деревьев твердолиственных и мягколиственных, 
3. количество деревьев по группам (диаметр от 1 до 10, от 11 до 20 и тд), 
4. является ли дерево многоствольным, число стволов, 
5. диаметр пня. Если указно несколько диаметров, берем бОльший. Диаметр пня у многоствольных больше. 
6. Количество вывозимых порубочных остатков

### Кустарники

In [405]:
def chop_bush_counter(df):
  try:
    df_bush = df[(~df.bush.isna()) & (df.action == 'Вырубить')]\
          [['type', 'bush', 'height', 'description']]
    df_bush['bush_cat'] = df['type'].apply(lambda x: '1поросль' if x == 'поросль' else '2куст')
    df_bush_to_chop = df_bush.groupby('bush_cat', as_index=False)[['bush']].sum()
    return df_bush_to_chop
  except:
    print('Ошибочка')

In [406]:
df_bush_to_chop = chop_bush_counter(df)

In [407]:
df_bush_to_chop

,bush_cat,bush
0,2куст,20.0


In [408]:
try: 
    bush_weigth = df_bush_to_chop.bush[0]*0.05 + df_bush_to_chop.bush[1]*0.08
except:
    try:
        bush_weigth = df_bush_to_chop.bush[0]*0.05
    except:
        bush_weigth = df_bush_to_chop.bush[1]*0.08

In [409]:
print(f'Общее количество кустов {df_bush_to_chop.bush.sum()}, вес порубочных остатков {bush_weigth} т.')

Общее количество кустов 20.0, вес порубочных остатков 1.0 т.


### Деревья

Сначала выделим некоторые вспомогательные функции: деление на группы, подсчет ствольности, деление на категории по твердости древесины, и впомогательные данные по весу в тоннах относительно диаметра ствола

In [410]:
#ствольность обычно указана в комментарии
def is_mnogostv(row):
  for i in range(5):
    if str(i)+' с' in row:
        return int(i)
  return 1   

In [411]:
def tree_type(row):
  tverd_types = ['Клен', 'Бук', 'Орех', 'Ясень' , 'Яблоня', 'Вишня', 
           'Сухостой', 'Самосев', 'Рябина', 'Акация']
  for tree_type in tverd_types:
    if tree_type.lower() in row.lower():
      return 'Твердолиственное'
  return 'Мягколиственное/хвойное'

In [412]:
def diameter_bins(row):
  return f'до {row // 10 + 1}0'

In [413]:
df_mass = {'до 10' : 0.0794285714285714, 
           'до 20' : 0.277714285714286, 
           'до 30' : 0.763809523809524, 
           'до 40' : 1.53942857142857, 
           'до 50' : 2.67476190476191, 
           'до 60' : 4.01542857142857, 
           'до 70' : 6.57371428571429, 
           'до 80' : 8.21714285714286, 
           'до 90' : 10.6822857142857, 
           'поросль' : 0.05, 
           'куст' : 0.01}

In [414]:
def the_last_stv(row):
    try: 
        result = int(str(row).split('-')[-1])
    except:
        result = int(str(row).split(',')[-1])
    return result

А теперь собственно функция, которая посчитает нам итоговую таблицу c количеством деревьев по категориям и диаметром и количеством порубочных остатков (а также диаметры пней для корчевки).

In [415]:
df_tree = df[(df.bush.isna()) & (df.action == 'Вырубить')]\
              [['type', 'tree', 'diameter', 'description']]

In [416]:
df[(df.bush.isna()) & (df.action == 'Вырубить')]

,number,type,tree,bush,diameter,height,description,action
0,1044,Береза,1.0,NaN,16,NaN,сухостой,Вырубить
2,1067,Сосна,1.0,NaN,66,NaN,no info,Вырубить
3,1068,Сосна,1.0,NaN,48,NaN,многочисл.сухие ветви,Вырубить
4,1069,Береза,1.0,NaN,46,NaN,no info,Вырубить
5,1070,Сосна,1.0,NaN,58,NaN,сухобочина,Вырубить
...,...,...,...,...,...,...,...,...
710,3314,Ель,1.0,NaN,14,NaN,no info,Вырубить
721,3322,Ель,1.0,NaN,36,NaN,no info,Вырубить
722,3323,Ель,1.0,NaN,38,NaN,no info,Вырубить
723,3324,Сосна,1.0,NaN,42,NaN,no info,Вырубить


In [417]:
df_tree['stvol'] = df.description.apply(is_mnogostv)

In [418]:
df_tree['diameter'] = df_tree['diameter'].apply(the_last_stv)

In [419]:
df_tree[df_tree.description != 'no info']

,type,tree,diameter,description,stvol
0,Береза,1.0,16,сухостой,1
3,Сосна,1.0,48,многочисл.сухие ветви,1
5,Сосна,1.0,58,сухобочина,1
12,Клен остролистный,1.0,10,спилы,1
30,Дуб,1.0,16,многочисл. сухие ветви,1
34,Дуб,1.0,6,от пня,1
35,Сосна,1.0,48,сухобочина,1
187,Липа,1.0,46,искривл.,1
191,Сосна,1.0,60,ствол наклонен,1
193,Сосна,1.0,64,"ствол наклонен, многочисл.сухие ветви",1


In [334]:
df_tree['stvol_count'] = df_tree['stvol'] * df_tree['tree']

In [335]:
df_tree['stump_diameter'] = df_tree['diameter'] * df_tree['stvol'] + 5

In [336]:
df_tree['tree_type'] = df_tree['type'].apply(tree_type)

In [337]:
df_tree['diameter_bins'] = df_tree['diameter'].apply(diameter_bins)

In [338]:
df_tree['stump_diameter_bins'] = df_tree['stump_diameter'].apply(diameter_bins)

In [339]:
df_tree['weight'] = df_tree['diameter_bins'].map(df_mass) * df_tree['stvol_count']

In [340]:
df_tree

,type,tree,diameter,description,stvol,stvol_count,stump_diameter,tree_type,diameter_bins,stump_diameter_bins,weight
0,Сосна,1.0,56,ствол наклонен,1,1.0,61,Мягколиственное/хвойное,до 60,до 70,4.015429
1,Сосна,1.0,40,no info,1,1.0,45,Мягколиственное/хвойное,до 50,до 50,2.674762
5,Сосна,2.0,28,no info,1,2.0,33,Мягколиственное/хвойное,до 30,до 40,1.527619
6,Сосна,1.0,58,no info,1,1.0,63,Мягколиственное/хвойное,до 60,до 70,4.015429
7,Сосна,1.0,10,ствол наклонен,1,1.0,15,Мягколиственное/хвойное,до 20,до 20,0.277714
...,...,...,...,...,...,...,...,...,...,...,...
1184,Береза,1.0,40,no info,1,1.0,45,Мягколиственное/хвойное,до 50,до 50,2.674762
1186,Ель,1.0,8,no info,1,1.0,13,Мягколиственное/хвойное,до 10,до 20,0.079429
1187,Береза,1.0,46,no info,1,1.0,51,Мягколиственное/хвойное,до 50,до 60,2.674762
1199,Рябина,1.0,6,no info,1,1.0,11,Твердолиственное,до 10,до 20,0.079429


In [341]:
df_tree_grouped = df_tree.groupby(['tree_type', 'diameter_bins'])\
              [['tree', 'stvol_count', 'weight']].sum()

In [342]:
df_tree_grouped

tree  stvol_count      weight
tree_type               diameter_bins                                
Мягколиственное/хвойное до 10           41.0         54.0    4.289143
                        до 20           96.0         99.0   27.493714
                        до 30          113.0        115.0   87.838095
                        до 40          138.0        144.0  221.677714
                        до 50          146.0        155.0  414.588095
                        до 60          149.0        158.0  634.437714
                        до 70           43.0         44.0  289.243429
                        до 80            4.0          4.0   32.868571
                        до 90            5.0          5.0   53.411429
Твердолиственное        до 10           31.0         42.0    3.336000
                        до 20           38.0         41.0   11.386286
                        до 30           12.0         14.0   10.693333
                        до 40            3.0          3.0    4.618286
                        до 50            2.0          2.0    5.349524
                        до 60            2.0          2.0    8.030857
                        до 70            1.0          1.0    6.573714

In [343]:
df_tree_trees = (pd.concat([df_tree_grouped,
           df_tree_grouped.groupby(level=0).sum()
           .assign(claim_type= "total")
           .set_index('claim_type', append=True)]).sort_index()
           .rename(columns={'tree':'Кол-во деревьев', 
                            'stvol_count': 'Кол-во стволов',
                            'weight': 'Тоннаж'}))

In [344]:
df_tree_trees

Кол-во деревьев  Кол-во стволов  \
tree_type               diameter_bins                                    
Мягколиственное/хвойное total                    735.0           778.0   
                        до 10                     41.0            54.0   
                        до 20                     96.0            99.0   
                        до 30                    113.0           115.0   
                        до 40                    138.0           144.0   
                        до 50                    146.0           155.0   
                        до 60                    149.0           158.0   
                        до 70                     43.0            44.0   
                        до 80                      4.0             4.0   
                        до 90                      5.0             5.0   
Твердолиственное        total                     89.0           105.0   
                        до 10                     31.0            42.0   
                        до 20                     38.0            41.0   
                        до 30                     12.0            14.0   
                        до 40                      3.0             3.0   
                        до 50                      2.0             2.0   
                        до 60                      2.0             2.0   
                        до 70                      1.0             1.0   

                                            Тоннаж  
tree_type               diameter_bins               
Мягколиственное/хвойное total          1765.847905  
                        до 10             4.289143  
                        до 20            27.493714  
                        до 30            87.838095  
                        до 40           221.677714  
                        до 50           414.588095  
                        до 60           634.437714  
                        до 70           289.243429  
                        до 80            32.868571  
                        до 90            53.411429  
Твердолиственное        total            49.988000  
                        до 10             3.336000  
                        до 20            11.386286  
                        до 30            10.693333  
                        до 40             4.618286  
                        до 50             5.349524  
                        до 60             8.030857  
                        до 70             6.573714

In [420]:
def chop_tree_counter(df):
  try:
    df_tree = df[(df.bush.isna()) & (df.action == 'Вырубить')]\
              [['type', 'tree', 'diameter', 'description']]
    df_tree['stvol'] = df.description.apply(is_mnogostv)
    df_tree['diameter'] = df_tree['diameter'].apply(the_last_stv)
    df_tree['stvol_count'] = df_tree['stvol'] * df_tree['tree']
    df_tree['stump_diameter'] = df_tree['diameter'] * df_tree['stvol'] + 1
    df_tree['tree_type'] = df_tree['type'].apply(tree_type)
    df_tree['diameter_bins'] = df_tree['diameter'].apply(diameter_bins)
    df_tree['stump_diameter_bins'] = df_tree['stump_diameter'].apply(diameter_bins)
    df_tree['weight'] = df_tree['diameter_bins'].map(df_mass)
    df_tree_grouped = df_tree.groupby(['tree_type', 'diameter_bins'])\
              [['tree', 'stvol_count', 'weight']].sum()
    df_tree_trees = (pd.concat([df_tree_grouped,
           df_tree_grouped.groupby(level=0).sum()
           .assign(claim_type= "total")
           .set_index('claim_type', append=True)]).sort_index()
           .rename(columns={'tree':'Кол-во деревьев', 
                            'stvol_count': 'Кол-во стволов',
                            'weight': 'Тоннаж'}))
    df_tree_grouped_diam = df_tree.groupby('diameter_bins')\
              [['tree', 'stvol_count', 'weight']].sum()
    
    df_stumps = (df_tree.groupby(['tree_type', 'stump_diameter_bins'])[['tree']].sum()
                .rename(columns={'tree':'Кол-во пней'}))
    return df_tree_trees, df_stumps, df_tree_grouped_diam
  except:
    print('Ошибочка')

In [421]:
trees, stumps, trees2 = chop_tree_counter(df)

In [422]:
trees

Кол-во деревьев  Кол-во стволов  \
tree_type               diameter_bins                                    
Мягколиственное/хвойное total                     91.0            91.0   
                        до 10                      7.0             7.0   
                        до 20                     16.0            16.0   
                        до 30                      9.0             9.0   
                        до 40                     16.0            16.0   
                        до 50                     20.0            20.0   
                        до 60                     13.0            13.0   
                        до 70                      6.0             6.0   
                        до 80                      3.0             3.0   
                        до 90                      1.0             1.0   
Твердолиственное        total                      6.0             6.0   
                        до 10                      1.0             1.0   
                        до 20                      5.0             5.0   

                                           Тоннаж  
tree_type               diameter_bins              
Мягколиственное/хвойное total          216.976381  
                        до 10            0.556000  
                        до 20            4.443429  
                        до 30            6.874286  
                        до 40           24.630857  
                        до 50           53.495238  
                        до 60           52.200571  
                        до 70           39.442286  
                        до 80           24.651429  
                        до 90           10.682286  
Твердолиственное        total            1.785714  
                        до 10            0.397143  
                        до 20            1.388571

In [429]:
trees['Кол-во деревьев'].sum() - 91-6


97.0

In [430]:
trees2

,tree,stvol_count,weight
diameter_bins,,,
до 10,8.0,8.0,0.953143
до 20,21.0,21.0,5.832000
до 30,9.0,9.0,6.874286
до 40,16.0,16.0,24.630857
до 50,20.0,20.0,53.495238
до 60,13.0,13.0,52.200571
до 70,6.0,6.0,39.442286
до 80,3.0,3.0,24.651429
до 90,1.0,1.0,10.682286


In [431]:
trees2.stvol_count.sum()

97.0

In [432]:
trees2.tree.sum()

97.0

In [433]:
tree_weigh = trees.query('diameter_bins == "total"')['Тоннаж'].sum()

In [434]:
print(f'''Итого вывозимый тоннаж порубочных остатков 
      {tree_weigh.round(2)} от деревьев, 
      {bush_weigth} от кустов, 
      всего {round(tree_weigh + bush_weigth, 2)} тонны''')

Итого вывозимый тоннаж порубочных остатков 
      218.76 от деревьев, 
      1.0 от кустов, 
      всего 219.76 тонны


In [435]:
stumps

Кол-во пней
tree_type               stump_diameter_bins             
Мягколиственное/хвойное до 10                        7.0
                        до 20                       16.0
                        до 30                        9.0
                        до 40                       16.0
                        до 50                       20.0
                        до 60                       13.0
                        до 70                        6.0
                        до 80                        3.0
                        до 90                        1.0
Твердолиственное        до 10                        1.0
                        до 20                        5.0